In [ ]:
import cv2
import numpy as np
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pywhatkit as kit
from joblib import load  # Import the load function
import pygame  # Alarm sound
import gspread
from google.oauth2.service_account import Credentials

# Initialize pygame for alarm sound
pygame.mixer.init()
alarm_sound = r"A:\Final Yr Project\fire_alarm.mp3"  # Path to alarm sound file

# Alarm function
def play_alarm():
    pygame.mixer.music.load(alarm_sound)
    pygame.mixer.music.play()

# Email alert function
def send_alert(subject, body):
    email_user = 'manohar62415@gmail.com'
    email_password = 'XXXX XXXX XXXX XXXX'  # Use environment variables for security
    email_send = 'manohar62415@gmail.com'

    msg = MIMEMultipart()
    msg['From'] = email_user
    msg['To'] = email_send
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(email_user, email_password)
            server.sendmail(email_user, email_send, msg.as_string())
            print("Email alert sent successfully!")
    except Exception as e:
        print(f"Failed to send email alert: {e}")

# WhatsApp alert function
def send_whatsapp_alert(message):
    try:
        # Get the current time and schedule the message 1 minute ahead
        current_hour = time.localtime().tm_hour
        current_minute = time.localtime().tm_min + 1  # Next minute
        phone_number = "+91XXXXXXXXXX"  # Replace with the recipient's number

        # Send the WhatsApp message
        kit.sendwhatmsg(phone_number, message, current_hour, current_minute)
        print("WhatsApp alert sent successfully!")
    except Exception as e:
        print(f"Failed to send WhatsApp alert: {e}")


# Google Sheets setup
SERVICE_ACCOUNT_FILE = r"A:\Final Yr Project\celtic-origin-449104-d8-b659e79168b1.json"  # Path to your JSON key file
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
gc = gspread.authorize(credentials)
sheet_url = "https://docs.google.com/spreadsheets/d/1WQB1dCF4oS5-y69GBQtLFjp2aCDkl4DOvJxdyoKBMQA/edit?gid=0#gid=0"  # Replace with your Google Sheet URL
sheet = gc.open_by_url(sheet_url).sheet1  # Opens the first sheet

# Function to log events
def log_event(event_type, details):
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    sheet.append_row([timestamp, event_type, details])
    print(f"Event logged: {event_type} - {details}")

# Load YOLO model
yolo_net = cv2.dnn.readNet(
    r'A:\Final Yr Project\yolov3.cfg.txt',
    r'A:\Final Yr Project\yolov3.weights'
)
with open(r'A:\Final Yr Project\coco.names') as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

# Initialize video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise Exception("Could not open video source")

# Parameters
CROWD_THRESHOLD = 5
UNATTENDED_THRESHOLD = 60
FRAME_AVERAGE_COUNT = 10

# Tracking variables
bag_trackers = {}
bag_last_seen_time = {}
frame_counts = []

# Fire detection function
def detect_fire(frame):
    # Convert to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define fire color range in HSV
    lower_fire = np.array([5, 150, 150])
    upper_fire = np.array([15, 255, 255])

    # Mask the image to detect fire-related colors
    mask = cv2.inRange(hsv, lower_fire, upper_fire)
    fire_detected = cv2.countNonZero(mask)

    if fire_detected > 1000:  # Threshold for fire detection
        return True
    return False

# Real-time video analysis with bounding boxes and confidence scores
print("Starting real-time monitoring...")
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame")
        break

    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outputs = yolo_net.forward(output_layers)

    height, width, _ = frame.shape
    boxes = []
    class_ids = []
    confidences = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:  # Only consider objects with confidence > 0.3
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = center_x - w // 2
                y = center_y - h // 2
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.3, 0.4)

    if indices is not None and len(indices) > 0:
        indices = indices.flatten()
        people_count = 0

        for i in indices:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]

            # Draw bounding box and label with confidence score
            color = (0, 255, 0)  # Green color for the box
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = f"{label}: {confidence:.2f}"
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            if class_ids[i] == 0:  # 'person' class
                people_count += 1

            elif class_ids[i] == 67:  # 'backpack' class
                bag_trackers[i] = (x, y, w, h)
                bag_last_seen_time[i] = time.time()

        if people_count > CROWD_THRESHOLD:
            event_message = "Crowd detected at Railway Station. Immediate attention required."
            log_event("Crowd Detected", event_message)
            send_alert("Crowd Detected", event_message)
            send_whatsapp_alert(event_message)
            play_alarm()

        for bag_id, last_seen in bag_last_seen_time.items():
            if time.time() - last_seen > UNATTENDED_THRESHOLD:
                event_message = "Unattended bag detected at Railway Station."
                log_event("Unattended Bag", event_message)
                send_alert("Unattended Bag Detected", event_message)
                send_whatsapp_alert(event_message)
                play_alarm()

        # Enhanced fire detection
        if detect_fire(frame):
            event_message = "Fire detected at Railway Station. Immediate attention required."
            log_event("Fire Detected", event_message)
            send_alert("Fire Detected", event_message)
            send_whatsapp_alert(event_message)
            play_alarm()
            

    # Display the frame with bounding boxes and labels
    
    cv2.imshow("Railway Station Safety Monitoring", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html
Starting real-time monitoring...
Event logged: Fire Detected - Fire detected at Railway Station. Immediate attention required.
Failed to send email alert: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d2e1a72fcca58-74af56d91d7sm1461543b3a.141 - gsmtp')
In 8 Seconds WhatsApp will open and after 15 Seconds Message will be Delivered!
WhatsApp alert sent successfully!


KeyboardInterrupt: 

: 